In [1]:
import json
import os
import re
import xml.dom.minidom
from xml.dom.minidom import parse, parseString
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
replace_elements = ["</TEXT>","<TEXT>","[","]","•","\n"]

In [3]:
txt_path = "PathologyDataset/pathology-report-information-extraction/Data/Pathologytxt/"
xml_path = "PathologyDataset/pathology-report-information-extraction/Data/Finalxml/"

In [4]:
all_tags = []
txt_list = os.listdir(txt_path)
xml_list = os.listdir(xml_path)
for ix, xml_file in enumerate(xml_list):
    with open(xml_path+xml_file, "r", encoding="utf8") as f:
        xml_data = f.read() #.replace('<?xml version="1.0" encoding="UTF-8" ?>\n\n', '')
        all_tags.extend([tag.split()[0].replace("<","")for tag in re.findall("<\w+\sid",xml_data)])
all_tags=list(set(all_tags))

In [5]:
all_tags

['Fuhrman_Nuclear_Grade',
 'Pateint_ID',
 'Lymph_Nodes',
 'Tnm_Histologic_Grade',
 'Lymph_Nodes_Invasion',
 'Histologic_Type',
 'Lymph_Vascular_Invasion',
 'Renal_Vein_Invasion',
 'Anatomical_Position',
 'Tumor_Diameter',
 'AJCC_Classification']

In [40]:
def replacer(text):
    for el in replace_elements:
        text = text.replace(el," ")
    finding = re.findall("TCGA[-\w+]+", text)
    if finding:
        text = text.replace(finding[0]," ")
    text = re.sub(" +"," ", text).strip().lower()
    return text

In [41]:
all_entities=["id","spans","text","type","comment"]

In [42]:
import itertools
struct_data = {"_".join(el):[] for el in itertools.product(*[all_tags,all_entities])}
all_tags.append("TEXT")
struct_data["TEXT"]=[]
struct_data

{'Renal_Vein_Invasion_id': [],
 'Renal_Vein_Invasion_spans': [],
 'Renal_Vein_Invasion_text': [],
 'Renal_Vein_Invasion_type': [],
 'Renal_Vein_Invasion_comment': [],
 'Lymph_Nodes_Invasion_id': [],
 'Lymph_Nodes_Invasion_spans': [],
 'Lymph_Nodes_Invasion_text': [],
 'Lymph_Nodes_Invasion_type': [],
 'Lymph_Nodes_Invasion_comment': [],
 'Tumor_Diameter_id': [],
 'Tumor_Diameter_spans': [],
 'Tumor_Diameter_text': [],
 'Tumor_Diameter_type': [],
 'Tumor_Diameter_comment': [],
 'Fuhrman_Nuclear_Grade_id': [],
 'Fuhrman_Nuclear_Grade_spans': [],
 'Fuhrman_Nuclear_Grade_text': [],
 'Fuhrman_Nuclear_Grade_type': [],
 'Fuhrman_Nuclear_Grade_comment': [],
 'Lymph_Vascular_Invasion_id': [],
 'Lymph_Vascular_Invasion_spans': [],
 'Lymph_Vascular_Invasion_text': [],
 'Lymph_Vascular_Invasion_type': [],
 'Lymph_Vascular_Invasion_comment': [],
 'Pateint_ID_id': [],
 'Pateint_ID_spans': [],
 'Pateint_ID_text': [],
 'Pateint_ID_type': [],
 'Pateint_ID_comment': [],
 'AJCC_Classification_id': [],
 '

In [47]:
def fill_data(xml_data):
    bs_data = BeautifulSoup(xml_data, 'xml') 
    for tag in all_tags:
        tag_string = bs_data.find_all(tag)
        if tag != "TEXT":
            for entity in all_entities:                
                if not tag_string:
                    struct_data[tag+"_"+entity].append([None])
                else:
                    entity_string = [tag_str.get(entity) for tag_str in tag_string]
                    struct_data[tag+"_"+entity].append(entity_string)
        else:
            struct_data[tag].append(str(tag_string))

In [48]:
for ix, xml_file in enumerate(xml_list):
    with open(xml_path+xml_file, "r", encoding="utf8") as f:
        xml_data = f.read() #.replace('<?xml version="1.0" encoding="UTF-8" ?>\n\n', '')
        fill_data(xml_data)

In [49]:
df = pd.DataFrame(struct_data)

In [50]:
df["TEXT"] = df["TEXT"].apply(replacer)

In [51]:
col_ordered = [
    'Pateint_ID_text',
    'Pateint_ID_id',
    'Pateint_ID_spans',
    'Pateint_ID_type',
    'Pateint_ID_comment',
    'TEXT',
    'AJCC_Classification_id',
    'AJCC_Classification_spans',
    'AJCC_Classification_text',
    'AJCC_Classification_type',
    'AJCC_Classification_comment',
    'Anatomical_Position_id',
    'Anatomical_Position_spans',
    'Anatomical_Position_text',
    'Anatomical_Position_type',
    'Anatomical_Position_comment',
    'Histologic_Type_id',
    'Histologic_Type_spans',
    'Histologic_Type_text',
    'Histologic_Type_type',
    'Histologic_Type_comment',
    'Lymph_Nodes_Invasion_id',
    'Lymph_Nodes_Invasion_spans',
    'Lymph_Nodes_Invasion_text',
    'Lymph_Nodes_Invasion_type',
    'Lymph_Nodes_Invasion_comment',
    'Lymph_Vascular_Invasion_id',
    'Lymph_Vascular_Invasion_spans',
    'Lymph_Vascular_Invasion_text',
    'Lymph_Vascular_Invasion_type',
    'Lymph_Vascular_Invasion_comment',
    'Lymph_Nodes_id',
    'Lymph_Nodes_spans',
    'Lymph_Nodes_text',
    'Lymph_Nodes_type',
    'Lymph_Nodes_comment',
    'Fuhrman_Nuclear_Grade_id',
    'Fuhrman_Nuclear_Grade_spans',
    'Fuhrman_Nuclear_Grade_text',
    'Fuhrman_Nuclear_Grade_type',
    'Fuhrman_Nuclear_Grade_comment',
    'Renal_Vein_Invasion_id',
    'Renal_Vein_Invasion_spans',
    'Renal_Vein_Invasion_text',
    'Renal_Vein_Invasion_type',
    'Renal_Vein_Invasion_comment',
    'Tnm_Histologic_Grade_id',
    'Tnm_Histologic_Grade_spans',
    'Tnm_Histologic_Grade_text',
    'Tnm_Histologic_Grade_type',
    'Tnm_Histologic_Grade_comment',
    'Tumor_Diameter_id',
    'Tumor_Diameter_spans',
    'Tumor_Diameter_text',
    'Tumor_Diameter_type',
    'Tumor_Diameter_comment',
]

In [53]:
df =df[col_ordered]

In [55]:
df.to_excel("PathologyDataset/structured_sampledataset.xlsx", index=False)